In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import datetime

from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.config import INDICATORS

# 1. Data Wrangle

In [2]:
TRAIN_START_DATE = '2020-01-01'
TRAIN_END_DATE = '2020-10-01'
TRADE_START_DATE = '2021-10-01'
TRADE_END_DATE = '2021-12-31'

symbols = [
    'BTC-USD',
    'ETH-USD',
    'USDT-USD',
    'BNB-USD',
    'XRP-USD',
    'SOL-USD',
    'DOGE-USD'
]

In [3]:
# TODO: use Binance for higher resolution data
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader

df_raw = YahooDownloader(
    start_date = TRAIN_START_DATE,
    end_date = TRADE_END_DATE,
    ticker_list = symbols).fetch_data()

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

Shape of DataFrame:  (5010, 8)


In [4]:
df_raw.head()

,date,open,high,low,close,volume,tic,day
0,2020-01-01,13.730962,13.873946,13.654942,13.689083,172980718,BNB-USD,2
1,2020-01-01,7194.892090,7254.330566,7174.944336,7200.174316,18565664997,BTC-USD,2
2,2020-01-01,0.002028,0.002052,0.002021,0.002033,51180941,DOGE-USD,2
3,2020-01-01,129.630661,132.835358,129.198288,130.802002,7935230330,ETH-USD,2
4,2020-01-01,0.999571,1.006873,0.994924,0.999836,21503143454,USDT-USD,2


# Preprocess Data

In [5]:
fe = FeatureEngineer(
    use_technical_indicator=True,
    tech_indicator_list = INDICATORS,
    use_vix=True,
    use_turbulence=True,
    user_defined_feature=False)
processed = fe.preprocess_data(df_raw)

[*********************100%%**********************]  1 of 1 completed

Successfully added technical indicators
Shape of DataFrame:  (503, 8)
Successfully added vix


Successfully added turbulence index


In [6]:
processed.sample(5)

,date,open,high,low,close,volume,tic,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
2632,2021-09-28,1.000461,1.000839,1.000089,1.000298,66373042997,USDT-USD,1,0.000058,1.001149,0.999474,49.648675,8.288201,8.505671,1.000311,1.000370,23.250000,0.857945
2648,2021-10-01,0.204427,0.224607,0.201994,0.223010,1435469263,DOGE-USD,4,-0.015081,0.258875,0.184723,46.407834,-61.404033,19.639277,0.239858,0.258971,21.100000,3.639011
439,2020-04-17,7116.552734,7167.183105,7050.332031,7096.184570,32513423567,BTC-USD,4,33.379509,7525.739363,6164.819622,48.861196,91.553264,0.672650,6685.716553,7394.056958,38.150002,0.000000
2136,2021-06-02,362.360870,418.665131,350.956787,401.262329,4486850584,BNB-USD,2,-47.368550,592.453219,196.187480,48.217678,-41.784957,19.091366,472.624957,496.276307,17.480000,1.522098
2664,2021-10-06,442.406250,442.479309,415.667725,435.401367,2006344571,BNB-USD,2,4.524209,461.230330,314.112736,53.963482,84.819060,2.003693,396.684233,421.376298,21.000000,4.640698


In [7]:
import itertools

list_ticker = processed['tic'].unique().tolist()
list_date = list(pd.date_range(processed['date'].min(),processed['date'].max()).strftime('%Y-%m-%d'))
#list_date
combination = list(itertools.product(list_date, list_ticker))
combination
processed_full = pd.DataFrame(combination, columns=['date', 'tic']).merge(processed)
processed_full = processed_full[processed_full['date'].isin(processed['date'])]
processed_full = processed_full.fillna(0)
processed_full.sample(5)

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
2352,2021-07-23,BNB-USD,293.601318,300.993774,281.696106,299.282990,1849700307,4,-8.343657,335.229220,275.736884,46.055664,-38.914501,21.081625,301.220141,326.472543,17.200001,0.737415
902,2020-08-06,DOGE-USD,0.003517,0.003636,0.003461,0.003560,62944678,3,0.000115,0.003630,0.003039,56.800686,37.633930,45.041119,0.003413,0.002937,22.650000,0.000000
192,2020-02-19,BNB-USD,24.074766,24.331917,22.215130,22.333511,298250332,2,1.475201,28.269995,16.339986,59.717923,52.048746,16.605669,20.760635,18.655989,14.380000,0.000000
1925,2021-04-12,XRP-USD,1.354004,1.470145,1.324077,1.467735,13216559312,0,0.215262,1.447483,0.155245,75.743241,214.862921,87.584176,0.697981,0.596036,16.910000,34.179386
745,2020-06-30,BTC-USD,9185.581055,9217.835938,9084.837891,9137.993164,15735797744,1,-72.969738,9682.566656,9018.137348,50.007062,-116.012538,23.268076,9489.227214,9382.829232,30.430000,0.000000


# Data Split and Save

In [8]:
train = data_split(processed_full, TRAIN_START_DATE, TRAIN_END_DATE)
trade = data_split(processed_full, TRADE_START_DATE, TRADE_END_DATE)

print(len(train))
print(len(trade))

1134
372


In [9]:
train.to_parquet("./train.parquet")
print(f"train.shape: {train.shape}")

train.shape: (1134, 18)


In [10]:
trade.to_parquet("./trade.parquet")
print(f"trade.shape: {trade.shape}")

trade.shape: (372, 18)


In [11]:
stock_dimension = len(processed_full.tic.unique())
state_space = 1  + 2*stock_dimension + len(INDICATORS)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 6, State Space: 61


# Train, Test, and Make Env

## Design Environment

We'll be modeling our environment as a stock environment because the CryptoEnv built into FinRL Meta doesn't integrate well with other FinRL pipelines. Also, from the Agent's perspective the statespace, actions, and rewards are the same as a stock environment.

TODO: try out the cryptoenv, by fixing ta-lib requirement

In [12]:
stock_dimension = len(processed_full.tic.unique())
state_space = 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 6, State Space: 61


In [13]:
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv

buy_cost_list = sell_cost_list = [0.001] * stock_dimension
num_stock_shares = [0] * stock_dimension

env_kwargs = {
    "hmax": 100,
    "initial_amount": 1_000_000,
    "num_stock_shares": num_stock_shares,
    "buy_cost_pct": buy_cost_list,
    "sell_cost_pct": sell_cost_list,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4,
}

e_train_gym = StockTradingEnv(df = train, **env_kwargs)

In [14]:
env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))

<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>


# Initialize Agent & Train

In [15]:
from finrl.agents.stablebaselines3.models import DRLAgent

agent = DRLAgent(env = env_train)

# Set the corresponding values to 'True' for the algorithms that you want to use
if_using_a2c = True
if_using_ddpg = False
if_using_ppo = True
if_using_td3 = False
if_using_sac = False

In [16]:
from stable_baselines3.common.logger import configure
from finrl.config import TRAINED_MODEL_DIR, RESULTS_DIR

model_a2c = agent.get_model("a2c")

if if_using_a2c:
    # set up logger
    tmp_path = RESULTS_DIR + '/a2c'
    new_logger_a2c = configure(tmp_path, ['stdout', 'csv', 'tensorboard'])
    # set new logger
    model_a2c.set_logger(new_logger_a2c)

{'n_steps': 5, 'ent_coef': 0.01, 'learning_rate': 0.0007}
Using cuda device
Logging to results/a2c


In [17]:
trained_a2c = agent.train_model(
    model=model_a2c,
    tb_log_name='a2c',
    total_timesteps=50000) if if_using_a2c else None

--------------------------------------
| time/                 |            |
|    fps                | 404        |
|    iterations         | 100        |
|    time_elapsed       | 1          |
|    total_timesteps    | 500        |
| train/                |            |
|    entropy_loss       | -8.53      |
|    explained_variance | -0.231     |
|    learning_rate      | 0.0007     |
|    n_updates          | 99         |
|    policy_loss        | -19.4      |
|    reward             | -1.1550831 |
|    std                | 1          |
|    value_loss         | 5.35       |
--------------------------------------
------------------------------------
| time/                 |          |
|    fps                | 452      |
|    iterations         | 200      |
|    time_elapsed       | 2        |
|    total_timesteps    | 1000     |
| train/                |          |
|    entropy_loss       | -8.56    |
|    explained_variance | 0.00759  |
|    learning_rate      | 0.0007   |
|    n

In [18]:
trained_a2c.save(TRAINED_MODEL_DIR + '/agent_a2c') if if_using_a2c else None

# Testing

In [19]:
stock_dimension = len(trade.tic.unique())
state_space = 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 6, State Space: 61


In [20]:
buy_cost_list = sell_cost_list = [0.001] * stock_dimension
num_stock_shares = [0] * stock_dimension

env_kwargs = {
    "hmax": 100,
    "initial_amount": 1_000_000,
    "num_stock_shares": num_stock_shares,
    "buy_cost_pct": buy_cost_list,
    "sell_cost_pct": sell_cost_list,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4,
}

In [21]:
e_trade_gym = StockTradingEnv(
    df = trade,
    turbulence_threshold = 70,
    risk_indicator_col = 'turbulence',
    **env_kwargs
)

In [22]:
df_account_value_a2c, df_actions_a2c = DRLAgent.DRL_prediction(
    model=trained_a2c,
    environment = e_trade_gym
) if if_using_a2c else (None, None)

hit end!


# 3. Backtesting

In [23]:
df_result_a2c = df_account_value_a2c.set_index(df_account_value_a2c.columns[0])
result = pd.DataFrame()
if if_using_a2c:
    result = pd.merge(result, df_result_a2c, how='outer', left_index = True, right_index=True)

In [24]:
col_name = []
col_name.append('A2C') if if_using_a2c else None
result.columns = col_name

In [25]:
plt.rcParams['figure.figsize'] = (15,5)
result.plot()

<Axes: xlabel='date'>

# Mean Variance Optimization

In [26]:
# Helps us process data into a form for weight calculation
def process_df_for_mvo(df):
    df = df.sort_values(['date', 'tic'], ignore_index=True)[['date', 'tic', 'close']]
    fst = df
    fst = fst.iloc[0:stock_dimension, :]
    tic = fst['tic'].tolist()

    mvo = pd.DataFrame()

    for k in range(len(tic)):
        mvo[tic[k]] = 0

    for i in range(df.shape[0]//stock_dimension):
        n = df
        n = n.iloc[i*stock_dimension:(i+1)*stock_dimension, :]
        date = n['date'][i*stock_dimension]
        mvo.loc[date] = n['close'].tolist()

    return mvo

def StockReturnsComputing(StockPrice, Rows, Columns):
    import numpy as np
    StockReturn = np.zeros([Rows-1, Columns])
    for j in range(Columns):                     # j: Assets
        for i in range(Rows-1):                  # i: Daily Prices
            StockReturn[i,j]=((StockPrice[i+1, j]-StockPrice[i,j])/StockPrice[i,j]) * 100
    return StockReturn

In [27]:
StockData = process_df_for_mvo(train)
TradeData = process_df_for_mvo(trade)

TradeData.to_numpy()

#compute asset returns
arStockPrices = np.asarray(StockData)
[Rows, Cols]=arStockPrices.shape
arReturns = StockReturnsComputing(arStockPrices, Rows, Cols)

# compute mean returns and variance covariance matrix of returns
meanReturns = np.mean(arReturns, axis=0)
covReturns = np.cov(arReturns, rowvar=False)

# set precision for printing results
np.set_printoptions(precision=3, suppress=True)

# display mean returns and variance-covariance matrix of returns
print('Mean returns of assets in k-portfolio 1\n', meanReturns)
print('Variance-Covariance matrix of returns\n', covReturns)

Mean returns of assets in k-portfolio 1
 [0.64  0.353 0.322 0.769 0.003 0.272]
Variance-Covariance matrix of returns
 [[37.826 22.943 20.533 30.237 -1.621 23.699]
 [22.943 22.104 16.743 26.114 -1.267 19.52 ]
 [20.533 16.743 38.63  21.936 -1.093 20.577]
 [30.237 26.114 21.936 39.412 -1.64  28.297]
 [-1.621 -1.267 -1.093 -1.64   0.539 -1.179]
 [23.699 19.52  20.577 28.297 -1.179 26.437]]


In [28]:
from pypfopt.efficient_frontier import EfficientFrontier

ef_mean = EfficientFrontier(meanReturns, covReturns, weight_bounds=(0, 0.5))
raw_weights_mean = ef_mean.max_sharpe()
cleaned_weights_mean = ef_mean.clean_weights()
mvo_weights = np.array([1_000_000 * cleaned_weights_mean[i] for i in range(6)])

In [29]:
LastPrice = np.array([1/p for p in StockData.tail(1).to_numpy()[0]])
Initial_Portfolio = np.multiply(mvo_weights, LastPrice)

Portfolio_Assets = TradeData @ Initial_Portfolio
MVO_result = pd.DataFrame(Portfolio_Assets, columns=["Mean Var"])

MVO_result

,Mean Var
2021-10-01,5.561047e+06
2021-10-04,5.656177e+06
2021-10-05,5.866209e+06
2021-10-06,5.914494e+06
2021-10-07,5.933265e+06
...,...
2021-12-22,6.673126e+06
2021-12-23,6.862875e+06
2021-12-27,6.824954e+06
2021-12-28,6.470923e+06


In [30]:
df_result_a2c = df_account_value_a2c.set_index(df_account_value_a2c.columns[0])

result = pd.DataFrame()
if if_using_a2c:
    result = pd.merge(result, df_result_a2c, how='outer', left_index=True, right_index=True)

In [31]:
col_name = []
col_name.append('A2C') if if_using_a2c else None
result.columns = col_name

In [32]:
result = pd.merge(result, MVO_result, how='outer', left_index=True, right_index=True)
result

,A2C,Mean Var
date,,
2021-10-01,1.000000e+06,5.561047e+06
2021-10-04,1.019275e+06,5.656177e+06
2021-10-05,1.068781e+06,5.866209e+06
2021-10-06,1.145085e+06,5.914494e+06
2021-10-07,1.114261e+06,5.933265e+06
...,...,...
2021-12-22,1.018842e+06,6.673126e+06
2021-12-23,1.063268e+06,6.862875e+06
2021-12-27,1.061570e+06,6.824954e+06


In [33]:
plt.rcParams['figure.figsize'] = (15,5)
plt.figure()
result.plot()

<Axes: xlabel='date'>